In [1]:
#https://pypi.org/project/oauth2client/
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#from df2gspread import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

## Read google sheets

Define the scope

In [3]:
# define the scope
#SCOPE = ['https://spreadsheets.google.com/feeds',
#    "https://www.googleapis.com/auth/drive"]
SCOPE = ["https://www.googleapis.com/auth/drive"]
API_FILE = "gcp_api/n2n-project-credentials.json"

Credentials

In [4]:
# add credentials to the account
credentials = ServiceAccountCredentials.from_json_keyfile_name(API_FILE, SCOPE)

# authorize the clientsheet
client = gspread.authorize(credentials)

# get the instance of the Spreadsheet
sheet = client.open('Copy of N2N - Database')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

In [5]:
# get the total number of columns
print(sheet_instance.col_count)
print(sheet_instance.row_count)

16
4995


Database

In [6]:
data = sheet_instance.get_all_values()
dfToUpdate = pd.DataFrame(data[1:], columns=data[0])

In [7]:
dfToUpdate

,#,Date,City,Season,Industry / Event,Format,Attendance,Email,First Name,Last Name,Country of Origin,Area of Expertise,Employment Status,Employer,Dream Job,Linkedin
0,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,fabysantanar@gmail.com,Fabiana,Santana,Venezuela,"Employee Experience, Business Partner",Employed,Spin Master,,
1,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,davhd21@gmail.com,David,Herrera,Mexico,Talent,Employed,Korn Ferry,,
2,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,karla.cadario@wilsonhcg.com,Karla,Cadario Moreno,Bolivia,Reclutamiento y Seleccion,Employed,WilsonHCG,,
3,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,eri_mabu@hotmail.com,Erika,Martínez,Colombia,Recruiting,Employed,Accenture,,
4,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,padronalfredo@outlook.com,Alfredo,Padron,Venezuela,Tech,Employed,Scotiabank,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,177,"November 27, 2023",Toronto,9,Legal,Online,Attending,mfpineror@gmail.com,Maria Fernanda,Pineros Rojas,Colombianone,Human Rightsnone,Unemployed and looking for opportunitiesnone,Nonnone,Work At Ngonone,Maria Fernanda Piñeros Rojasnone
4990,177,"November 27, 2023",Toronto,9,Legal,Online,Checked In,cp_abi_98@hotmail.com,Abigail,Condori Pereyra,Perunone,Criminalnone,Employed and looking for opportunitiesnone,Dominion Pardons And Waiversnone,.None,Https://Www.Linkedin.Com/In/Abigailcondoripere...
4991,177,"November 27, 2023",Toronto,9,Legal,Online,Attending,cvilches.guzman@gmail.com,carolina,vilches,Chilenone,Clean Servicesnone,Unemployed and looking for opportunitiesnone,Sunshinenone,Achieve Confidence In My Professional Fieldnone,Nonenone
4992,177,"November 27, 2023",Toronto,9,Legal,Online,Not Attending,hbatchounr@gmail.com,Helen,Batchoun,Nonenone,Nonenone,Nonenone,Nonenone,Nonenone,Nonenone


Define actual meeting number

In [8]:
dfToUpdate['#'] = dfToUpdate['#'].astype(int)

In [9]:
MAX_NUMBER = dfToUpdate['#'].max()
MAX_NUMBER

177

In [10]:
dfToUpdate['Date']

0        January 14, 2021
1        January 14, 2021
2        January 14, 2021
3        January 14, 2021
4        January 14, 2021
              ...        
4989    November 27, 2023
4990    November 27, 2023
4991    November 27, 2023
4992    November 27, 2023
4993    November 27, 2023
Name: Date, Length: 4994, dtype: object

In [11]:
#dfToUpdate['Date'] = pd.to_datetime(dfToUpdate['Date'], format='%B %d, %Y')
#dfToUpdate['Date'] = dfToUpdate['Date'].dt.strftime('%Y-%m-%d')
#dfToUpdate['Date'] = pd.to_datetime(dfToUpdate['Date'], format='%Y-%m-%d')

In [12]:
#dfToUpdate['Date'].max()

In [13]:
#dfToUpdate['Date'].max() + timedelta(days=1)

## Read new data

In [14]:
#df = pd.read_excel('gcp_files/report-2023-11-06T2158.xlsx')
#df = pd.read_excel('gcp_files/report-2023-11-07T1311.xlsx')
#df = pd.read_excel('gcp_files/report-2023-11-13T2113.xlsx')
#df = pd.read_excel('gcp_files/report-2023-11-20T1926.xlsx')
df = pd.read_excel('gcp_files/test.xlsx')


#### City

In [15]:
def extract_city(event_name):
    if "Montreal" in event_name:
        return "Montreal"
    else:
        return "Toronto"

In [16]:
df['City'] = df['Event Name'].apply(extract_city)

#### Seasons


Toronto

In [17]:
def return_season_toronto(meeting_date):
    if meeting_date <= datetime(2021,3,4): return 1
    elif meeting_date <= datetime(2021,6,10): return 2
    elif meeting_date <= datetime(2021,12,9): return 3
    elif meeting_date <= datetime(2022,4,14): return 4
    elif meeting_date <= datetime(2022,7,28): return 5
    elif meeting_date <= datetime(2022,11,7): return 6
    elif meeting_date <= datetime(2023,4,6): return 7
    elif meeting_date <= datetime(2023,6,29): return 8
    elif meeting_date <= datetime(2023,12,31): return 9
    else: return 9


Montreal

In [18]:
def return_season_montreal(meeting_date):
    if meeting_date <= datetime(2021,3,4):
        return 1
    else:
        return 1

Apply according to city fucntion

In [19]:
'''df['Date'] = pd.to_datetime(df['Date Attending'], format='%b %d, %Y at %I:%M %p')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')'''

#date_attending = datetime.strptime(date_attending, "%Y-%m-%dT%H:%M:%S")
    #date_attending = date_attending.strftime("%Y-%m-%d")

"df['Date'] = pd.to_datetime(df['Date Attending'], format='%b %d, %Y at %I:%M %p')\ndf['Date'] = df['Date'].dt.strftime('%Y-%m-%d')\ndf['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')"

In [20]:

df['Date'] = pd.to_datetime(df['Date Attending'], format='%Y-%m-%dT%H:%M:%S')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [21]:
def season_based_on_city(df):
    if df['City'] == 'Toronto':
        return return_season_toronto(df['Date'])
    elif df['City'] == 'Montreal':
        return return_season_montreal(df['Date'])
    else:
        # Handle other cities or cases if needed
        return None

In [22]:
# Apply the custom function to create a new 'Season' column
df['Season'] = df.apply(season_based_on_city, axis=1)

#### Date

In [23]:
df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')
#df['Date']

#### Industry / Event

In [24]:
def extractName(eventName):
    if "|" in(eventName): # or " | " in(eventName):
        #if " | " in(eventName):
        #    split_result = eventName.split(' | ')
        #else:
        split_result = eventName.split('|')

        eventNameBefore = split_result[0]
        eventNameAfter = split_result[1]

        if "NotWorking to Networking" in(eventNameBefore) or "NotWorking2Networking" in(eventNameBefore):
            eventName = eventNameAfter
        elif "NotWorking to Networking" in(eventNameAfter) or "NotWorking2Networking" in(eventNameAfter) or "N2N Montreal" in(eventNameAfter) or "N2N" in(eventNameAfter) or 'Not working to Networking Montreal' in(eventNameAfter) :
            eventName = eventNameBefore

        if 'Latinos in ' in eventName:
            split_result = eventName.split('Latinos in ')
            return split_result[1].strip()
        else:
            return eventName.strip()
    else:
        return eventName.strip()

In [25]:
df['Industry / Event'] = df['Event Name'].apply(extractName)

In [26]:
df['Industry / Event'].value_counts()

Industry / Event
Entrepreneurship           70
Design & Graphic Design    41
Sales                      23
Fashion                    16
Name: count, dtype: int64

In [27]:
replacementsIndustryEvent = {
    'Art &amp; Entertainment' : 'Art & Entertainment',
    'Banca, Contabilidad &amp; Finanzas' : 'Banking, Finance & Insurance',
    'Social Work &amp; Mental Health' : 'Social Work & Mental Health',
    'Hospitality &amp; Tourism': 'Hospitality & Tourism',
    'Social Work &amp; Mental Health': 'Social Work & Mental Health',
    'Design &amp; Graphic Design' : 'Design & Graphic Design'
}


In [28]:
df['Industry / Event']=df['Industry / Event'].replace(replacementsIndustryEvent)

In [29]:
df['Industry / Event'].unique()

array(['Design & Graphic Design', 'Sales', 'Entrepreneurship', 'Fashion'],
      dtype=object)

#### Format

In [30]:
def returnFormat(eventName):
    return "In Person" if "In Person" in eventName else "Online"

df['Format'] = df['Event Name'].apply(returnFormat)

#### \#

In [31]:
MAX_NUMBER

177

In [32]:
# Create an empty list to store the counter values
counter_values = []

# Initialize variables to keep track of the previous date and city
prev_date = None
prev_city = None
counter = 0

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    current_date = row['Date']
    current_city = row['City']

    # Check if either the date or city has changed from the previous row
    if current_date != prev_date or current_city != prev_city:
        counter += 1
    counter_values.append(counter)

    # Update the previous date and city values for the next iteration
    prev_date = current_date
    prev_city = current_city

In [33]:
df['#'] = counter_values
df['#'] = df['#'] + MAX_NUMBER

#### Attendance


In [34]:
df['Attendance'] = df['Attendee Status']

In [35]:
df['Attendance'].unique()

array(['Attending', 'Checked In', 'Not Attending'], dtype=object)

#### Add columns

In [64]:
def add_cols (df,field1, field2,result_field):
    # Check if the columns exist in the DataFrame
    if field1 in df.columns and field2 in df.columns:
        # If both columns exist, perform the operations
        df[field1] = df[field1].astype(str)
        df[field2] = df[field2].astype(str)

        df[field1].replace('nan','', inplace=True)
        df[field2].replace('nan','', inplace=True)
        
        #df[field1].fillna('', inplace=True)
        #df[field2].fillna('', inplace=True)

        # Concatenate and strip
        df[result_field] = (df[field1] +
                                    df[field2]).str.strip()

        df[result_field] = df[result_field].str.title()

    elif field1 in df.columns:
        df[result_field] = df[field1].astype(str)
        df[result_field] = df[result_field].str.title()

    elif field2 in df.columns:
        df[result_field] = df[field2].astype(str)
        df[result_field] = df[result_field].str.title()
    else:
        # Handle the case where one or both columns are missing
        # You can choose to do nothing, create a default value, or perform any other desired action.
        pass  # or do something else

#### Country of Origin

In [65]:
add_cols(df,'What country are you from in Latin America? (if applicable)','¿De qué país eres en América Latina? (si aplica)','Country of Origin')

In [38]:
'''# Check for missing values and replace them with an empty string
df['What country are you from in Latin America? (if applicable)'].fillna('', inplace=True)
df['¿De qué país eres en América Latina? (si aplica)'].fillna('', inplace=True)

# Concatenate and strip
df['Country of Origin'] = (df['What country are you from in Latin America? (if applicable)'] +
                            df['¿De qué país eres en América Latina? (si aplica)']).str.strip()

df['Country of Origin'] = df['Country of Origin'].str.title()'''

"# Check for missing values and replace them with an empty string\ndf['What country are you from in Latin America? (if applicable)'].fillna('', inplace=True)\ndf['¿De qué país eres en América Latina? (si aplica)'].fillna('', inplace=True)\n\n# Concatenate and strip\ndf['Country of Origin'] = (df['What country are you from in Latin America? (if applicable)'] +\n                            df['¿De qué país eres en América Latina? (si aplica)']).str.strip()\n\ndf['Country of Origin'] = df['Country of Origin'].str.title()"

In [66]:
# Define a dictionary for replacements
replacements = {
    '': np.nan,
    'Canadá': 'Canada',
    'Perú': 'Peru',
    'España': 'Spain',
    'México': 'Mexico',
    'República Dominicana': 'Dominican Republic',
    '-': np.nan,
    'Not': np.nan,
    'nan': np.nan,
    'Alberta, British Columbia And Calgary.' : 'Canada',
    'Brasil': 'Brazil',
    'Amazonia' : 'Colombia',
    'Dr': np.nan,
    'Yes': np.nan,
    'Spain/Colombia': 'Colombia',
    'X' : np.nan,
    'Na,India': np.nan,
    'None (Spain)': np.nan,
    'India':np.nan,
    'South Africa':np.nan,
    'Puebla, México' : 'Mexico',
    'Ciudad De México' : 'Mexico',
    'Coló Me La' : np.nan,
    'Born In Canada.': np.nan,
    'Mex': np.nan
}

In [67]:
df['Country of Origin']=df['Country of Origin'].replace(replacements)

In [68]:
df['Country of Origin'].unique()

array([nan, 'Colombia', 'Ecuador', 'Bolivia', 'Chile', 'Brazil', 'Canada',
       'Mexico', 'Peru', 'Venezuela', 'Honduras', 'Costa Rica',
       'Guatemala', 'Puerto Rico', 'Argentina'], dtype=object)

#### Area of Expertise

In [42]:
add_cols(df,'What area/subject do you specialize in? (in this industry)','¿En qué área/materia te especializas? (en esta industria)','Area of Expertise')

In [43]:
'''# Check for missing values and replace them with an empty string
df['What area/subject do you specialize in? (in this industry)'].fillna('', inplace=True)
df['¿En qué área/materia te especializas? (en esta industria)'].fillna('', inplace=True)

df['Area of Expertise'] = (df['What area/subject do you specialize in? (in this industry)'] +
                            df['¿En qué área/materia te especializas? (en esta industria)']).str.strip()'''

"# Check for missing values and replace them with an empty string\ndf['What area/subject do you specialize in? (in this industry)'].fillna('', inplace=True)\ndf['¿En qué área/materia te especializas? (en esta industria)'].fillna('', inplace=True)\n\ndf['Area of Expertise'] = (df['What area/subject do you specialize in? (in this industry)'] +\n                            df['¿En qué área/materia te especializas? (en esta industria)']).str.strip()"

#### Employment Status

In [44]:
add_cols(df,"What\'s your employment status?",'¿Cuál es tu situación laboral?','Employment Status')

In [45]:
'''# Check for missing values and replace them with an empty string
df["What\'s your employment status?"].fillna('', inplace=True)
df['¿Cuál es tu situación laboral?'].fillna('', inplace=True)

df['Employment Status'] = (df["What\'s your employment status?"] +
                            df['¿Cuál es tu situación laboral?'])'''

'# Check for missing values and replace them with an empty string\ndf["What\'s your employment status?"].fillna(\'\', inplace=True)\ndf[\'¿Cuál es tu situación laboral?\'].fillna(\'\', inplace=True)\n\ndf[\'Employment Status\'] = (df["What\'s your employment status?"] +\n                            df[\'¿Cuál es tu situación laboral?\'])'

In [46]:
# Define a dictionary for replacements
replacementsEmploymentStatus = {
    'Empleado': 'Employed',
    'Empleado en búsqueda de nuevas oportunidades': 'Employed and looking for opportunities',
    'Desempleado y buscando oportunidades': 'Unemployed and looking for opportunities',
    'Empleado y en búsqueda de oportunidades': 'Employed and looking for opportunities',
    '': np.nan
}

In [47]:
df['Employment Status']=df['Employment Status'].replace(replacementsEmploymentStatus)

In [48]:
df['Employment Status'] = df['Employment Status'].str.capitalize()

In [49]:
df['Employment Status'].unique()

array(['Employed and looking for opportunities', 'Employed',
       'Unemployed and looking for opportunities', nan], dtype=object)

#### Employer

In [50]:
add_cols(df,'If employed, what company do you work for?','Si estás empleado, ¿para qué empresa trabajas?','Employer')

In [51]:
'''# Check for missing values and replace them with an empty string
df['If employed, what company do you work for?'].fillna('', inplace=True)
df['Si estás empleado, ¿para qué empresa trabajas?'].fillna('', inplace=True)

df['Employer'] = df['If employed, what company do you work for?'] + df['Si estás empleado, ¿para qué empresa trabajas?']'''

"# Check for missing values and replace them with an empty string\ndf['If employed, what company do you work for?'].fillna('', inplace=True)\ndf['Si estás empleado, ¿para qué empresa trabajas?'].fillna('', inplace=True)\n\ndf['Employer'] = df['If employed, what company do you work for?'] + df['Si estás empleado, ¿para qué empresa trabajas?']"

#### Dream job

In [52]:
add_cols(df,'What is your dream job in Canada?','¿Cuál es tu trabajo soñado en Canadá?','Dream Job')

In [53]:
'''# Check for missing values and replace them with an empty string
df['What is your dream job in Canada?'].fillna('', inplace=True)
df['¿Cuál es tu trabajo soñado en Canadá?'].fillna('', inplace=True)

df['Dream Job'] = df['What is your dream job in Canada?'] + df['¿Cuál es tu trabajo soñado en Canadá?']'''

"# Check for missing values and replace them with an empty string\ndf['What is your dream job in Canada?'].fillna('', inplace=True)\ndf['¿Cuál es tu trabajo soñado en Canadá?'].fillna('', inplace=True)\n\ndf['Dream Job'] = df['What is your dream job in Canada?'] + df['¿Cuál es tu trabajo soñado en Canadá?']"

#### Linkedin


In [54]:
add_cols(df,'Provide your LinkedIn if you want to connect with others in this community!',
         '¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!',
         'Linkedin')

In [55]:
'''# Check for missing values and replace them with an empty string
df['Provide your LinkedIn if you want to connect with others in this community!'].fillna('', inplace=True)
df['¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!'].fillna('', inplace=True)

df['Linkedin'] = df['Provide your LinkedIn if you want to connect with others in this community!'] + df['¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!']'''

"# Check for missing values and replace them with an empty string\ndf['Provide your LinkedIn if you want to connect with others in this community!'].fillna('', inplace=True)\ndf['¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!'].fillna('', inplace=True)\n\ndf['Linkedin'] = df['Provide your LinkedIn if you want to connect with others in this community!'] + df['¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!']"

#### Select and organize fields

In [56]:
df.columns

Index(['Event Name', 'Date Attending', 'Attendee Status', 'Email',
       'First Name', 'Last Name',
       'What country are you from in Latin America? (if applicable)',
       '¿De qué país eres en América Latina? (si aplica)',
       'What area/subject do you specialize in? (in this industry)',
       '¿En qué área/materia te especializas? (en esta industria)',
       'What's your employment status?', '¿Cuál es tu situación laboral?',
       'If employed, what company do you work for?',
       'Si estás empleado, ¿para qué empresa trabajas?',
       'What is your dream job in Canada?',
       '¿Cuál es tu trabajo soñado en Canadá?',
       'Provide your LinkedIn if you want to connect with others in this community!',
       '¡Proporciona tu LinkedIn si quieres conectarte con otros en esta comunidad!',
       'City', 'Date', 'Season', 'Industry / Event', 'Format', '#',
       'Attendance', 'Country of Origin', 'Area of Expertise',
       'Employment Status', 'Employer', 'Dream Job', 

In [57]:
#df = df[['#','Date', 'City', 'Season', 'Industry / Event', 'Format','Attendance', 'Email', 'First Name', 'Last Name', 'Country of Origin', 'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job', 'Linkedin']]
df2 = df[['#','Date', 'City', 'Season', 'Industry / Event', 'Format', 'Attendance',  'Email', 'First Name', 'Last Name', 'Country of Origin', 'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job', 'Linkedin']]

In [58]:
df2

,#,Date,City,Season,Industry / Event,Format,Attendance,Email,First Name,Last Name,Country of Origin,Area of Expertise,Employment Status,Employer,Dream Job,Linkedin
0,178,11/15/2023,Toronto,9,Design & Graphic Design,In Person,Attending,nana_duran70@hotmail.com,Sandra,Duran,NaN,X,Employed and looking for opportunities,X,X,X
1,178,11/15/2023,Toronto,9,Design & Graphic Design,In Person,Checked In,melcatster@gmail.com,Melissa,Mora,Colombia,Graphic Design Marketing,Employed,Kasian Architecture,Self Employment With Multiple Income Sources,Https://Www.Linkedin.Com/In/Melissamorat
2,178,11/15/2023,Toronto,9,Design & Graphic Design,In Person,Checked In,melcatster@gmail.com,Melissa,Mora,Colombia,Graphic Design Marketing,Employed,Kasian Architecture,Self Employment With Multiple Income Sources,Https://Www.Linkedin.Com/In/Melissamorat
3,178,11/15/2023,Toronto,9,Design & Graphic Design,In Person,Attending,sdpozodc@gmail.com,Stephanie,Pozo,Ecuador,Content And Brand Design And Digital Content,Unemployed and looking for opportunities,,Have The Opportunity To Get Into The Industry,@Stephaniepozodelcorral
4,178,11/15/2023,Toronto,9,Design & Graphic Design,In Person,Checked In,sergiodja1791@gmail.com,Sergio,Jimenez,Colombia,Graphic Design,Unemployed and looking for opportunities,,Ux Ui,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,183,11/23/2023,Toronto,9,Fashion,In Person,Attending,licorneviolet18@gmail.com,Camila,Duque,Colombia,Print Design,Employed and looking for opportunities,Gertex,Print Designer,Https://Www.Linkedin.Com/In/Camila-Duque-B6640...
146,183,11/23/2023,Toronto,9,Fashion,In Person,Attending,samuelndias@gmail.com,Samuel,Dias,Brazil,Design,Employed and looking for opportunities,La,Designer,
147,183,11/23/2023,Toronto,9,Fashion,In Person,Attending,juandelgado96@hotmail.es,Juan,Delgado,Ecuador,Fashion Y Beauty Photography,Employed and looking for opportunities,Starbucks,Photographer,Https://Www.Linkedin.Com/In/Juan-Diego-Delgado...
148,183,11/23/2023,Toronto,9,Fashion,In Person,Not Attending,linaibanezc@gmail.com,Lina,Ibáñez,NaN,,NaN,,,


In [59]:
len(df2.columns)

16

In [60]:
df2.columns

Index(['#', 'Date', 'City', 'Season', 'Industry / Event', 'Format',
       'Attendance', 'Email', 'First Name', 'Last Name', 'Country of Origin',
       'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job',
       'Linkedin'],
      dtype='object')

In [61]:
df2.to_excel('clean_file2.xlsx', index=False)

In [62]:
dfToUpdate.shape[0]

4994

In [63]:
#set_with_dataframe(sheet_instance, df2, row=dfToUpdate.shape[0] + 2, include_column_header=False)